In [1]:
!pip install pymongo

^C


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from pymongo import MongoClient
from urllib.parse import quote_plus
import pandas as pd

ModuleNotFoundError: No module named 'pymongo'

In [ ]:


username = "mongoadmin"
password = "Abcd@1234"
database_name = "Dlaw"

# Escape username and password
escaped_username = quote_plus(username)
escaped_password = quote_plus(password)

# Construct the connection string
connection_string = "mongodb://{}:{}@localhost:27018/".format(escaped_username, escaped_password)

# Create MongoClient
client = MongoClient(connection_string)

# Access the database
db = client[database_name]

# Specify the collection name
collection_name = "pkkh"  # Replace with your actual collection name

# Access the collection
collection = db[collection_name]

# Query the collection and convert the results to a DataFrame
cursor = collection.find()  # You can add query conditions if needed
df = pd.DataFrame(list(cursor))

# Close the MongoDB connection
client.close()

# Now you can work with the DataFrame (df)
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.drop(["CustomerID"], axis = 1, inplace=True)

kd=sb.kdeplot(data=df['Age']);

In [ ]:
kd=sb.kdeplot(data=df["Spending Score (1-100)"]);

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
boxplot=sb.boxplot(y=df["Spending Score (1-100)"], color="red")

plt.subplot(1,2,2)
sb.boxplot(y=df["Annual Income (k$)"])
plt.show()

In [ ]:
genders = df.Gender.value_counts()
plt.figure(figsize=(10,4))
sb.barplot(x=genders.index, y=genders.values)
plt.show()

In [ ]:
age18_25 = df.Age[(df.Age <= 25) & (df.Age >= 18)]
age26_35 = df.Age[(df.Age <= 35) & (df.Age >= 26)]
age36_45 = df.Age[(df.Age <= 45) & (df.Age >= 36)]
age46_55 = df.Age[(df.Age <= 55) & (df.Age >= 46)]
age55above = df.Age[df.Age >= 56]

x = ["18-25","26-35","36-45","46-55","55+"]
y = [len(age18_25.values),len(age26_35.values),len(age36_45.values),len(age46_55.values),len(age55above.values)]

plt.figure(figsize=(15,6))
sb.barplot(x=x, y=y, palette="rocket")
plt.title("Number of Customer and Ages")
plt.xlabel("Age")
plt.ylabel("Number of Customer")
plt.show()

In [ ]:
ss1_20 = df["Spending Score (1-100)"][(df["Spending Score (1-100)"] >= 1) & (df["Spending Score (1-100)"] <= 20)]
ss21_40 = df["Spending Score (1-100)"][(df["Spending Score (1-100)"] >= 21) & (df["Spending Score (1-100)"] <= 40)]
ss41_60 = df["Spending Score (1-100)"][(df["Spending Score (1-100)"] >= 41) & (df["Spending Score (1-100)"] <= 60)]
ss61_80 = df["Spending Score (1-100)"][(df["Spending Score (1-100)"] >= 61) & (df["Spending Score (1-100)"] <= 80)]
ss81_100 = df["Spending Score (1-100)"][(df["Spending Score (1-100)"] >= 81) & (df["Spending Score (1-100)"] <= 100)]

ssx = ["1-20", "21-40", "41-60", "61-80", "81-100"]
ssy = [len(ss1_20.values), len(ss21_40.values), len(ss41_60.values), len(ss61_80.values), len(ss81_100.values)]

plt.figure(figsize=(15,6))
sb.barplot(x=ssx, y=ssy, palette="nipy_spectral_r")
plt.title("Spending Scores")
plt.xlabel("Score")
plt.ylabel("Number of Customer Having the Score")
plt.show()

In [ ]:
ai0_30 = df["Annual Income (k$)"][(df["Annual Income (k$)"] >= 0) & (df["Annual Income (k$)"] <= 30)]
ai31_60 = df["Annual Income (k$)"][(df["Annual Income (k$)"] >= 31) & (df["Annual Income (k$)"] <= 60)]
ai61_90 = df["Annual Income (k$)"][(df["Annual Income (k$)"] >= 61) & (df["Annual Income (k$)"] <= 90)]
ai91_120 = df["Annual Income (k$)"][(df["Annual Income (k$)"] >= 91) & (df["Annual Income (k$)"] <= 120)]
ai121_150 = df["Annual Income (k$)"][(df["Annual Income (k$)"] >= 121) & (df["Annual Income (k$)"] <= 150)]

aix = ["$ 0 - 30,000", "$ 30,001 - 60,000", "$ 60,001 - 90,000", "$ 90,001 - 120,000", "$ 120,001 - 150,000"]
aiy = [len(ai0_30.values), len(ai31_60.values), len(ai61_90.values), len(ai91_120.values), len(ai121_150.values)]

plt.figure(figsize=(15,6))
sb.barplot(x=aix, y=aiy, palette="Set2")
plt.title("Annual Incomes")
plt.xlabel("Income")
plt.ylabel("Number of Customer")
plt.show()

In [ ]:
df.iloc[:,1:]

In [ ]:
df.head()

In [ ]:
from sklearn.cluster import KMeans

wcss = []
K=10
for k in range(1, K+1):
    kmeans = KMeans(n_clusters=k, init="k-means++")
    kmeans.fit(df.iloc[:,2:])
    wcss.append(kmeans.inertia_)
plt.figure(figsize=(12,6))
plt.grid()
plt.plot(range(1, K+1),wcss, linewidth=2, color="red", marker ="8")
plt.xlabel("K Value")
plt.xticks(np.arange(1, K+1, 1))
plt.ylabel("WCSS")
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

km = KMeans(n_clusters=5)
clusters = km.fit_predict(df.iloc[:,2:])
df["label"] = clusters

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df.Age[df.label == 0], df["Annual Income (k$)"][df.label == 0], df["Spending Score (1-100)"][df.label == 0], c='blue', s=60)
ax.scatter(df.Age[df.label == 1], df["Annual Income (k$)"][df.label == 1], df["Spending Score (1-100)"][df.label == 1], c='red', s=60)
ax.scatter(df.Age[df.label == 2], df["Annual Income (k$)"][df.label == 2], df["Spending Score (1-100)"][df.label == 2], c='green', s=60)
ax.scatter(df.Age[df.label == 3], df["Annual Income (k$)"][df.label == 3], df["Spending Score (1-100)"][df.label == 3], c='orange', s=60)
ax.scatter(df.Age[df.label == 4], df["Annual Income (k$)"][df.label == 4], df["Spending Score (1-100)"][df.label == 4], c='purple', s=60)
ax.view_init(30, 185)
plt.xlabel("Age")
plt.ylabel("Annual Income (k$)")
ax.set_zlabel('Spending Score (1-100)')
plt.show()

In [ ]:
username = "mongoadmin"
password = "Abcd@1234"
database_name = "Dlaw"

# Escape username and password
escaped_username = quote_plus(username)
escaped_password = quote_plus(password)

# Construct the connection string
connection_string = "mongodb://{}:{}@localhost:27018/".format(escaped_username, escaped_password)

# Create MongoClient
client = MongoClient(connection_string)

# Access the database
db = client[database_name]

# Specify the collection name
collection_name = "wine"  # Replace with your actual collection name

# Access the collection
collection = db[collection_name]

# Query the collection and convert the results to a DataFrame
cursor = collection.find()  # You can add query conditions if needed
df = pd.DataFrame(list(cursor))

# Close the MongoDB connection
client.close()

# Now you can work with the DataFrame (df)
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
from sklearn.cluster import KMeans

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['color'])
wcss = []
K=10
for k in range(1, K+1):
    kmeans = KMeans(n_clusters=k, init="k-means++")
    kmeans.fit(df.loc[:,['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'quality', 'good', 'label']])
    wcss.append(kmeans.inertia_)
plt.figure(figsize=(12,6))
plt.grid()
plt.plot(range(1, K+1),wcss, linewidth=2, color="red", marker ="8")
plt.xlabel("K Value")
plt.xticks(np.arange(1, K+1, 1))
plt.ylabel("WCSS")
plt.show()

In [ ]:
km = KMeans(n_clusters=4)
clusters = km.fit_predict(df.loc[:,['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'quality', 'good', 'label']])
df["label"] = clusters

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df.Age[df.label == 0], df["Annual Income (k$)"][df.label == 0], df["Spending Score (1-100)"][df.label == 0], c='blue', s=60)
ax.scatter(df.Age[df.label == 1], df["Annual Income (k$)"][df.label == 1], df["Spending Score (1-100)"][df.label == 1], c='red', s=60)
ax.scatter(df.Age[df.label == 2], df["Annual Income (k$)"][df.label == 2], df["Spending Score (1-100)"][df.label == 2], c='green', s=60)
ax.scatter(df.Age[df.label == 3], df["Annual Income (k$)"][df.label == 3], df["Spending Score (1-100)"][df.label == 3], c='orange', s=60)
ax.scatter(df.Age[df.label == 4], df["Annual Income (k$)"][df.label == 4], df["Spending Score (1-100)"][df.label == 4], c='purple', s=60)
ax.view_init(30, 185)
plt.xlabel("Age")
plt.ylabel("Annual Income (k$)")
ax.set_zlabel('Spending Score (1-100)')
plt.show()